A réaliser : 
- Une analyse descriptive des données, y compris une explication du sens des colonnes gardées, des arguments derrière la suppression de lignes ou de colonnes, des statistiques descriptives et des visualisations pertinentes.

## Import des modules

In [380]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import r2_score, mean_absolute_error , root_mean_squared_error, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance
from sklearn.pipeline import Pipeline

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler,FunctionTransformer

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [381]:

bc_after_eda = pd.read_csv("data/bc_after_eda.csv", index_col='Unnamed: 0',dtype={'NumberofFloors':float})
bc_after_eda.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1630 entries, 0 to 3375
Data columns (total 41 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   DataYear                         1630 non-null   int64  
 1   BuildingType                     1630 non-null   object 
 2   PrimaryPropertyType              1630 non-null   object 
 3   CouncilDistrictCode              1630 non-null   int64  
 4   Neighborhood                     1630 non-null   object 
 5   Latitude                         1630 non-null   float64
 6   Longitude                        1630 non-null   float64
 7   YearBuilt                        1630 non-null   int64  
 8   NumberofBuildings                1630 non-null   float64
 9   NumberofFloors                   1630 non-null   float64
 10  PropertyGFATotal                 1630 non-null   int64  
 11  PropertyGFAParking               1630 non-null   int64  
 12  PropertyGFABuilding(s)   

In [382]:
select_columns = ['LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA', 'ListOfAllPropertyUseTypes', 'FirstUseType', 'SecondLargestPropertyUseType', 'YearBuilt', 'NumberofFloors', 'NumberofBuildings'
                  ,'Latitude', 'Longitude', 'Neighborhood', 'SiteEnergyUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)', 'SteamUse(kBtu)','CalcSEUI(kBtu/sf)']
bc_after_eda = bc_after_eda[select_columns]
bc_after_eda

,LargestPropertyUseTypeGFA,SecondLargestPropertyUseTypeGFA,ListOfAllPropertyUseTypes,FirstUseType,SecondLargestPropertyUseType,YearBuilt,NumberofFloors,NumberofBuildings,Latitude,Longitude,Neighborhood,SiteEnergyUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),SteamUse(kBtu),CalcSEUI(kBtu/sf)
0,88434.0,0.0,Hotel,Hotel,NaN,1927,12.0,1.0,47.61220,-122.33799,DOWNTOWN,7.226362e+06,3.946027e+06,1.276453e+06,2003882.00,81.714753
1,83880.0,15064.0,"Hotel, Parking, Restaurant",Hotel,Parking,1996,11.0,1.0,47.61317,-122.33393,DOWNTOWN,8.387933e+06,3.242851e+06,5.145082e+06,0.00,80.991184
2,756493.0,0.0,Hotel,Hotel,NaN,1969,41.0,1.0,47.61393,-122.33810,DOWNTOWN,7.258702e+07,4.952666e+07,1.493800e+06,21566554.00,95.952010
3,61320.0,0.0,Hotel,Hotel,NaN,1926,10.0,1.0,47.61412,-122.33664,DOWNTOWN,6.794584e+06,2.768924e+06,1.811213e+06,2214446.25,110.805349
4,123445.0,68009.0,"Hotel, Parking, Swimming Pool",Hotel,Parking,1980,18.0,1.0,47.61375,-122.34047,DOWNTOWN,1.417261e+07,5.368607e+06,8.803998e+06,0.00,74.026168
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,12294.0,0.0,Office,Office,NaN,1990,1.0,1.0,47.56722,-122.31154,GREATER DUWAMISH,8.497457e+05,5.242709e+05,3.254750e+05,0.00,69.118732
3372,16000.0,0.0,Other - Recreation,Other,NaN,2004,1.0,1.0,47.59625,-122.32283,DOWNTOWN,9.502762e+05,3.965461e+05,5.537300e+05,0.00,59.392262
3373,7583.0,5574.0,"Fitness Center/Health Club/Gym, Other - Recrea...",Other,Fitness Center/Health Club/Gym,1974,1.0,1.0,47.63644,-122.35784,MAGNOLIA / QUEEN ANNE,5.765898e+06,1.792159e+06,3.973739e+06,0.00,438.238048
3374,6601.0,6501.0,"Fitness Center/Health Club/Gym, Food Service, ...",Mixed Use Property,Fitness Center/Health Club/Gym,1989,1.0,1.0,47.52832,-122.32431,GREATER DUWAMISH,7.194712e+05,3.488702e+05,3.706010e+05,0.00,52.956808


## Feature Engineering

A réaliser : Enrichir le jeu de données actuel avec de nouvelles features issues de celles existantes. 

### Préparation des features pour la modélisation

A réaliser :
* Si ce n'est pas déjà fait, supprimer toutes les colonnes peu pertinentes pour la modélisation.
* Tracer la distribution de la cible pour vous familiariser avec l'ordre de grandeur. En cas d'outliers, mettez en place une démarche pour les supprimer.
* Débarrassez-vous des features redondantes en utilisant une matrice de corrélation.
* Réalisez différents graphiques pour comprendre le lien entre vos features et la target (boxplots, scatterplots, pairplot si votre nombre de features numériques n'est pas très élevé).
*  Séparez votre jeu de données en un Pandas DataFrame X (ensemble de feautures) et Pandas Series y (votre target).
* Si vous avez des features catégorielles, il faut les encoder pour que votre modèle fonctionne.

#### Modes énergétiques

In [383]:
bc_after_eda['UseGas'] = (bc_after_eda['NaturalGas(kBtu)'].notna()) & (bc_after_eda['NaturalGas(kBtu)'] != 0)
bc_after_eda['UseSteam'] = (bc_after_eda['SteamUse(kBtu)'].notna()) & (bc_after_eda['SteamUse(kBtu)'] != 0)
bc_after_eda['UseElectricity'] = (bc_after_eda['Electricity(kBtu)'].notna()) & (bc_after_eda['Electricity(kBtu)'] != 0)


#### distance du centre ville

In [384]:
def haversine(lat1, lon1, lat2, lon2):
    # Entrées: Series -> conversion séparée
    lat1 = np.radians(lat1)
    lon1 = np.radians(lon1)
    lat2 = np.radians(lat2)
    lon2 = np.radians(lon2)
    R = 3958.8
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c


In [385]:
seattle_lat, seattle_lon = 47.6085965, -122.5049456

bc_after_eda['CityDistance'] = haversine(
    bc_after_eda['Latitude'], bc_after_eda['Longitude'],
    seattle_lat, seattle_lon
).round(2)
bc_after_eda['CityDistance'].describe()

count    1630.000000
mean        8.663564
std         1.414665
min         4.960000
25%         7.800000
50%         8.360000
75%         9.430000
max        15.200000
Name: CityDistance, dtype: float64

#### Utilisation multiple

In [386]:
bc_after_eda['MultipleUseType'] = bc_after_eda['ListOfAllPropertyUseTypes'].str.count('s')+1
bc_after_eda['MultipleUseType'].value_counts()

MultipleUseType
1    967
2    530
3     93
4     32
6      3
7      2
5      2
8      1
Name: count, dtype: int64

In [387]:
bc_after_eda.columns

Index(['LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
       'ListOfAllPropertyUseTypes', 'FirstUseType',
       'SecondLargestPropertyUseType', 'YearBuilt', 'NumberofFloors',
       'NumberofBuildings', 'Latitude', 'Longitude', 'Neighborhood',
       'SiteEnergyUse(kBtu)', 'Electricity(kBtu)', 'NaturalGas(kBtu)',
       'SteamUse(kBtu)', 'CalcSEUI(kBtu/sf)', 'UseGas', 'UseSteam',
       'UseElectricity', 'CityDistance', 'MultipleUseType'],
      dtype='object')

In [388]:
bc_after_eda.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1630 entries, 0 to 3375
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   LargestPropertyUseTypeGFA        1630 non-null   float64
 1   SecondLargestPropertyUseTypeGFA  1630 non-null   float64
 2   ListOfAllPropertyUseTypes        1630 non-null   object 
 3   FirstUseType                     1630 non-null   object 
 4   SecondLargestPropertyUseType     847 non-null    object 
 5   YearBuilt                        1630 non-null   int64  
 6   NumberofFloors                   1630 non-null   float64
 7   NumberofBuildings                1630 non-null   float64
 8   Latitude                         1630 non-null   float64
 9   Longitude                        1630 non-null   float64
 10  Neighborhood                     1630 non-null   object 
 11  SiteEnergyUse(kBtu)              1630 non-null   float64
 12  Electricity(kBtu)        

## Split train/test

In [389]:
predict_values = ['LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA', 'FirstUseType','SecondLargestPropertyUseType', 'MultipleUseType', 'UseSteam', 'UseElectricity', 'UseGas',
       'NumberofFloors', 'NumberofBuildings', 'CityDistance', 'Neighborhood','YearBuilt']
X = bc_after_eda[predict_values]
target = 'CalcSEUI(kBtu/sf)'
y = bc_after_eda[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [390]:
print("Index X_train avant pipeline :", X_train.index.equals(y_train.index))


Index X_train avant pipeline : True


In [391]:
y.sort_values()

595       2.115986
2246      2.304459
380       3.346921
609       3.529496
378       3.598461
           ...    
601     465.516620
3171    593.627028
558     639.679483
809     696.735220
240     834.402500
Name: CalcSEUI(kBtu/sf), Length: 1630, dtype: float64

## Finalisation des features

In [392]:
# ========================
# ÉTAPE 1: PREPROCESSING PERSONNALISÉ 
# ========================

from ocde_p6.preprocessing.transformer import fix_floors_and_discretize

# ========================
# ÉTAPE 2: PIPELINE COMPLET
# ========================

# Colonnes après votre preprocessing
categorical_features = ['FirstUseType', 'SecondLargestPropertyUseType', 'PropertySize',
                       'Neighborhood','AgeCategory','EnergyEra','HeightCategory']  # Ajoutez vos autres catégories ici

numerical_features = ['LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA', 
                     'CityDistance', 'MultipleUseType', 'NumberofFloors','NumberofBuildings']

# Pipeline complet
full_pipeline = Pipeline([
    # Étape 1: Preprocessing personnalisé
    ('preprocessing', FunctionTransformer(fix_floors_and_discretize, validate=False)),
    
    # Étape 2: Encodage + Normalisation
    ('encoder', ColumnTransformer([
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), 
         categorical_features),
        ('num', StandardScaler(), numerical_features)
    ], remainder='passthrough'))
])

# ========================
# ÉTAPE 3: APPLICATION 
# ========================

# Fit sur train et transform train/test
X_train_transformed = full_pipeline.fit_transform(X_train)
X_test_transformed = full_pipeline.transform(X_test)

# ========================
# ÉTAPE 4: CRÉER VOS DataFrames _final
# ========================

# Récupérer les noms des colonnes
onehot = full_pipeline.named_steps['encoder'].named_transformers_['cat']
onehot_names = onehot.get_feature_names_out(categorical_features)
num_names = [f"scaled_{col}" for col in numerical_features]

# Colonnes restantes (passthrough)
all_cols_after_preprocessing = fix_floors_and_discretize(X_train).columns
remaining_cols = [col for col in all_cols_after_preprocessing 
                 if col not in categorical_features + numerical_features]

# Noms finaux
final_feature_names = list(onehot_names) + num_names + remaining_cols

# Vos DataFrames finaux
X_train_final = pd.DataFrame(X_train_transformed, columns=final_feature_names,index=X_train.index )
X_test_final = pd.DataFrame(X_test_transformed, columns=final_feature_names,index=X_test.index)

# Conversion en numérique
for col in X_train_final.columns:
    X_train_final[col] = pd.to_numeric(X_train_final[col], errors='coerce')
    X_test_final[col] = pd.to_numeric(X_test_final[col], errors='coerce')

X_train_final.drop(columns=['AgeProperty', 'YearBuilt', 'SumPropertyGFA'], inplace=True)
X_test_final.drop(columns=['AgeProperty', 'YearBuilt', 'SumPropertyGFA'], inplace=True)

print(f"✅ Pipeline terminé!")
print(f"Shape finale: Train {X_train_final.shape}, Test {X_test_final.shape}")


✅ Pipeline terminé!
Shape finale: Train (1304, 122), Test (326, 122)


In [393]:
print(list(X_train_final.columns))

['FirstUseType_Automobile Dealership', 'FirstUseType_Bank Branch', 'FirstUseType_Courthouse', 'FirstUseType_Data Center', 'FirstUseType_Distribution Center', 'FirstUseType_Financial Office', 'FirstUseType_Fire Station', 'FirstUseType_Fitness Center/Health Club/Gym', 'FirstUseType_Food Service', 'FirstUseType_Hospital', 'FirstUseType_Hotel', 'FirstUseType_K-12 School', 'FirstUseType_Laboratory', 'FirstUseType_Large Office', 'FirstUseType_Library', 'FirstUseType_Lifestyle Center', 'FirstUseType_Manufacturing/Industrial Plant', 'FirstUseType_Medical Office', 'FirstUseType_Mixed Use Property', 'FirstUseType_Movie Theater', 'FirstUseType_Museum', 'FirstUseType_Office', 'FirstUseType_Other', 'FirstUseType_Parking', 'FirstUseType_Performing Arts', 'FirstUseType_Personal Services (Health/Beauty, Dry Cleaning, etc)', 'FirstUseType_Police Station', 'FirstUseType_Pre-school/Daycare', 'FirstUseType_Prison/Incarceration', 'FirstUseType_Refrigerated Warehouse', 'FirstUseType_Repair Services (Vehicle

In [394]:
#scaled = ['scaled_LargestPropertyUseTypeGFA', 'scaled_SecondLargestPropertyUseTypeGFA', 'scaled_CityDistance', 'scaled_MultipleUseType', 'scaled_NumberofFloors', 'scaled_NumberofBuildings']
#X_train_final[scaled].corr()

In [395]:
print("Index X_train_final après pipeline :", X_train_final.index.equals(y_train.index))

Index X_train_final après pipeline : True


In [396]:
X_train_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1304 entries, 1161 to 1972
Columns: 122 entries, FirstUseType_Automobile Dealership to UseGas
dtypes: bool(3), float64(119)
memory usage: 1.2 MB


In [397]:
# CODE COMPARAISON DES MODELES
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
models = {
    'DummyRegressor': DummyRegressor(strategy='mean'),
    'LinearRegression': LinearRegression(),
    'SVR': SVR(),
    'GradientBoosting': GradientBoostingRegressor(random_state=42),
    'RandomForest': RandomForestRegressor(random_state=42, n_jobs=-1)
}

scoring = ['neg_root_mean_squared_error','r2','neg_mean_absolute_error','neg_mean_absolute_percentage_error']
cv_results = {}

print("=== COMPARAISON MODÈLES AVEC PIPELINE COMPLET ===")
for name, model in models.items():
    # Pipeline complet + modèle final
    full_estimator = Pipeline([
        ('preprocess', full_pipeline.named_steps['preprocessing']),
        ('encode_scale', full_pipeline.named_steps['encoder']),
        ('model', model)
    ])
    scores = cross_validate(full_estimator, X_train, y_train, 
                            cv=5, scoring=scoring, n_jobs=-1)
    cv_results[name] = {
        'RMSE': -scores[f"test_{scoring[0]}"].mean(),
        'R2': scores[f"test_{scoring[1]}"].mean(),
        'MAE': -scores[f"test_{scoring[2]}"].mean(),
        'MAPE': -scores[f"test_{scoring[3]}"].mean()
    }
    print(f"{name} → R²: {cv_results[name]['R2']:.3f}, RMSE: {cv_results[name]['RMSE']:.2f}, MAE: {cv_results[name]['MAE']:.2f}, MAPE: {cv_results[name]['MAPE']:.3f}")
    
    

=== COMPARAISON MODÈLES AVEC PIPELINE COMPLET ===


/usr/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)
/usr/lib/pyth

DummyRegressor → R²: -0.013, RMSE: 70.03, MAE: 43.43, MAPE: 1.154


/usr/lib/python3.13/multiprocessing/queues.py:120: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  return _ForkingPickler.loads(res)


LinearRegression → R²: 0.394, RMSE: 53.80, MAE: 33.46, MAPE: 0.733
SVR → R²: -0.098, RMSE: 73.06, MAE: 38.90, MAPE: 0.732
GradientBoosting → R²: 0.392, RMSE: 54.07, MAE: 32.36, MAPE: 0.767
RandomForest → R²: 0.325, RMSE: 55.90, MAE: 32.56, MAPE: 0.727


In [398]:
choice= "MAPE"
if choice == "R2":
    best_model_name = max(cv_results.keys(), key=lambda x: cv_results[x][choice])
else:
    best_model_name = min(cv_results.keys(), key=lambda x: cv_results[x][choice])
print(f"\n🏆 Meilleur modèle selon {choice}: {best_model_name}")


🏆 Meilleur modèle selon MAPE: RandomForest


### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn. 

In [372]:
# Supposons best_model_name défini précédemment
#best_model_name = "RandomForest"
print(f"=== OPTIMISATION de {best_model_name} ===")

if best_model_name == 'RandomForest':
    estimator = RandomForestRegressor(random_state=42, n_jobs=-1)
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

elif best_model_name == 'GradientBoosting':
    estimator = GradientBoostingRegressor(random_state=42)
    param_grid = {
        'n_estimators': [100, 200],
        'learning_rate': [0.1, 0.05, 0.01],
        'max_depth': [3, 5, 7],
        'subsample': [0.8, 1.0]
    }

elif best_model_name == 'LinearRegression':
    estimator = LinearRegression()
    param_grid = {
        'fit_intercept': [True, False],
        'positive': [False, True]
    }

elif best_model_name == 'SVR':
    estimator = SVR()
    param_grid = {
        'kernel': ['rbf', 'linear', 'poly'],
        'C': [0.1, 1, 10],
        'gamma': ['scale', 'auto']
    }

else:
    raise ValueError(f"Modèle non supporté : {best_model_name}")

# Lancer GridSearchCV
gs = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=1
)

gs.fit(X_train_final, y_train)

print("Meilleurs paramètres :", gs.best_params_)
print("Meilleur score CV (RMSE) :", -gs.best_score_)

# Évaluation finale
final_model = gs.best_estimator_
y_pred = final_model.predict(X_test_final)

print("\n=== PERFORMANCE FINALE sur TEST ===")
print(f"RMSE: {root_mean_squared_error(y_test, y_pred):.0f}")
print(f"R²: {r2_score(y_test, y_pred):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred):.0f}")
print(f"MAPE: {mean_absolute_percentage_error(y_test, y_pred):.4f}")


=== OPTIMISATION de SVR ===
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Meilleurs paramètres : {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Meilleur score CV (RMSE) : 63.968423565774174

=== PERFORMANCE FINALE sur TEST ===
RMSE: 51
R²: 0.3508
MAE: 30
MAPE: 0.4840


=== OPTIMISATION de SVR ===
Fitting 5 folds for each of 18 candidates, totalling 90 fits
Meilleurs paramètres : {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Meilleur score CV (RMSE) : 63.970048252105485

=== PERFORMANCE FINALE sur TEST ===
RMSE: 51
R²: 0.3508
MAE: 30
MAPE: 0.4840

In [239]:
if best_model_name in ['GradientBoosting','RandomForest']:
    best_model = gs.best_estimator_

    importances = best_model.feature_importances_
    feature_names = X_train_final.columns

    # Affichage du top 15 features les plus importantes
    sorted_idx = importances.argsort()[::-1]
    print("Top 30 features by importance:")
    for idx in sorted_idx[:30]:
        print(f"- {feature_names[idx]}: {importances[idx]:.4f}")

# Sauvegarde BentoML

In [238]:
import bentoml
bentoml.sklearn.save_model(
    name="building_energy_rf_pipeline",
    model=full_pipeline,
    signatures={"predict": {"batchable": True}},
    metadata={
        "author": "day811",
        "feature_names": list(feature_names)
    },
    
)

Model(tag="building_energy_rf_pipeline:m43ov5fj4gmxqlhq", path="/tmp/bentoml-model-building_energy_rf_pipeline-c_g3knuc")